# Aquire Data

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split

from env import host, user, password
import acquire
import prepare

In [2]:
sql_query = '''select *
    from  properties_2017
    join predictions_2017 using(parcelid)
    where transactiondate between "2017-05-01" and "2017-08-31"
        and propertylandusetypeid between 260 and 266
            or propertylandusetypeid between 273 and 279
            and not propertylandusetypeid = 274
        and unitcnt = 1;
    '''
connection = f'mysql+pymysql://{user}:{password}@{host}/zillow'
df = pd.read_sql(sql_query, connection)

# Lets Clean the Data

In [3]:
df.head()

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate
0,14634203,2026522,1.0,NaN,NaN,2.0,3.0,None,NaN,2.0,...,289445.0,2016.0,213516.0,3490.16,None,NaN,6.059001e+13,625,-0.061973,2017-08-04
1,11721753,616260,NaN,NaN,NaN,2.0,3.0,None,6.0,2.0,...,205123.0,2016.0,163175.0,2627.48,None,NaN,6.037220e+13,1018,0.017785,2017-07-21
2,11289917,2061546,1.0,NaN,NaN,2.0,3.0,None,6.0,2.0,...,136104.0,2016.0,27214.0,2319.90,Y,15.0,6.037901e+13,1248,-0.362001,2017-06-23
3,11637029,2554497,1.0,NaN,NaN,2.0,3.0,None,9.0,2.0,...,810694.0,2016.0,554156.0,9407.01,Y,13.0,6.037701e+13,1734,-0.093399,2017-07-28
4,11705026,1834372,NaN,NaN,NaN,1.0,2.0,None,6.0,1.0,...,35606.0,2016.0,23624.0,543.69,None,NaN,6.037232e+13,1772,-0.146056,2017-06-30


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38582 entries, 0 to 38581
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      38582 non-null  int64  
 1   id                            38582 non-null  int64  
 2   airconditioningtypeid         12224 non-null  float64
 3   architecturalstyletypeid      101 non-null    float64
 4   basementsqft                  27 non-null     float64
 5   bathroomcnt                   38582 non-null  float64
 6   bedroomcnt                    38582 non-null  float64
 7   buildingclasstypeid           0 non-null      object 
 8   buildingqualitytypeid         23706 non-null  float64
 9   calculatedbathnbr             38438 non-null  float64
 10  decktypeid                    331 non-null    float64
 11  finishedfloor1squarefeet      3338 non-null   float64
 12  calculatedfinishedsquarefeet  38474 non-null  float64
 13  f

In [5]:
df = df.dropna(axis=1,thresh=35000)

In [6]:
df.head()

,parcelid,id,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,latitude,...,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id,logerror,transactiondate
0,14634203,2026522,2.0,3.0,2.0,1125.0,1125.0,6059.0,2.0,33922098.0,...,1974.0,75929.0,289445.0,2016.0,213516.0,3490.16,6.059001e+13,625,-0.061973,2017-08-04
1,11721753,616260,2.0,3.0,2.0,1316.0,1316.0,6037.0,2.0,34033180.0,...,1923.0,41948.0,205123.0,2016.0,163175.0,2627.48,6.037220e+13,1018,0.017785,2017-07-21
2,11289917,2061546,2.0,3.0,2.0,1458.0,1458.0,6037.0,2.0,34686163.0,...,1970.0,108890.0,136104.0,2016.0,27214.0,2319.90,6.037901e+13,1248,-0.362001,2017-06-23
3,11637029,2554497,2.0,3.0,2.0,1766.0,1766.0,6037.0,2.0,34073222.0,...,1980.0,256538.0,810694.0,2016.0,554156.0,9407.01,6.037701e+13,1734,-0.093399,2017-07-28
4,11705026,1834372,1.0,2.0,1.0,1421.0,1421.0,6037.0,1.0,33999877.0,...,1911.0,11982.0,35606.0,2016.0,23624.0,543.69,6.037232e+13,1772,-0.146056,2017-06-30


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38582 entries, 0 to 38581
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      38582 non-null  int64  
 1   id                            38582 non-null  int64  
 2   bathroomcnt                   38582 non-null  float64
 3   bedroomcnt                    38582 non-null  float64
 4   calculatedbathnbr             38438 non-null  float64
 5   calculatedfinishedsquarefeet  38474 non-null  float64
 6   finishedsquarefeet12          38388 non-null  float64
 7   fips                          38582 non-null  float64
 8   fullbathcnt                   38438 non-null  float64
 9   latitude                      38582 non-null  float64
 10  longitude                     38582 non-null  float64
 11  propertycountylandusecode     38582 non-null  object 
 12  propertylandusetypeid         38582 non-null  float64
 13  r

- What I want to do on top of that:
    - Delete columns:
        - `calculatedbathnbr`
            - because we already have the finished number of bathrooms
        - `finishedsquarefeet12`
            - because we have our finished square feet already
        - `propertycountylandusecode`
            - because we have the id for this already
        - `propertyzoningdesc`
            - because it is not that helpful
        - `logerror`
            - because logerror is an unknown feature in data dictonary
        - `transactiondate`
            - because it is not that helpful
        - `yearbuilt`
            - because it would not be read as a year but as a numeric value
        - `taxvaluedollarcnt`
            - using this would be cheating
        - `landtaxvaluedollarcnt`
            - using this would be cheating
    - Rename columns:
        - `bathroomcnt` to "bathrooms"
        - `bedroomcnt` to "bedrooms"
        - `calculatedfinishedsquarefeet` to "house_square_feet"
        - `fullbathcnt` to "full_baths"
        - `regionidzip` to "zip_code"
        - `regionidcity` to "city"
        - `regionidcounty` to "county"

In [8]:
df = df.drop(['calculatedbathnbr', 'finishedsquarefeet12', 
             'propertycountylandusecode', 'logerror', 'transactiondate',  
             'yearbuilt', 'taxvaluedollarcnt', 'landtaxvaluedollarcnt'], axis=1)
df.head()

,parcelid,id,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,fullbathcnt,latitude,longitude,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,structuretaxvaluedollarcnt,assessmentyear,taxamount,censustractandblock,id
0,14634203,2026522,2.0,3.0,1125.0,6059.0,2.0,33922098.0,-117954970.0,266.0,6.059001e+07,18874.0,1286.0,96186.0,6.0,75929.0,2016.0,3490.16,6.059001e+13,625
1,11721753,616260,2.0,3.0,1316.0,6037.0,2.0,34033180.0,-118355396.0,261.0,6.037220e+07,12447.0,3101.0,95997.0,0.0,41948.0,2016.0,2627.48,6.037220e+13,1018
2,11289917,2061546,2.0,3.0,1458.0,6037.0,2.0,34686163.0,-118113100.0,261.0,6.037901e+07,5534.0,3101.0,97318.0,0.0,108890.0,2016.0,2319.90,6.037901e+13,1248
3,11637029,2554497,2.0,3.0,1766.0,6037.0,2.0,34073222.0,-118390965.0,266.0,6.037701e+07,10389.0,3101.0,96086.0,0.0,256538.0,2016.0,9407.01,6.037701e+13,1734
4,11705026,1834372,1.0,2.0,1421.0,6037.0,1.0,33999877.0,-118291863.0,261.0,6.037232e+07,12447.0,3101.0,96018.0,0.0,11982.0,2016.0,543.69,6.037232e+13,1772


In [9]:
df = df.rename(columns={'bathroomcnt':'bathrooms', 'bedroomcnt':'bedrooms', 
                       'calculatedfinishedsquarefeet':'square_feet', 
                       'fullbathcnt':'full_baths', 'regionidzip':'zip_code', 
                       'regionidcity':'city', 'regionidcounty':'county'})
df.head()

,parcelid,id,bathrooms,bedrooms,square_feet,fips,full_baths,latitude,longitude,propertylandusetypeid,rawcensustractandblock,city,county,zip_code,roomcnt,structuretaxvaluedollarcnt,assessmentyear,taxamount,censustractandblock,id
0,14634203,2026522,2.0,3.0,1125.0,6059.0,2.0,33922098.0,-117954970.0,266.0,6.059001e+07,18874.0,1286.0,96186.0,6.0,75929.0,2016.0,3490.16,6.059001e+13,625
1,11721753,616260,2.0,3.0,1316.0,6037.0,2.0,34033180.0,-118355396.0,261.0,6.037220e+07,12447.0,3101.0,95997.0,0.0,41948.0,2016.0,2627.48,6.037220e+13,1018
2,11289917,2061546,2.0,3.0,1458.0,6037.0,2.0,34686163.0,-118113100.0,261.0,6.037901e+07,5534.0,3101.0,97318.0,0.0,108890.0,2016.0,2319.90,6.037901e+13,1248
3,11637029,2554497,2.0,3.0,1766.0,6037.0,2.0,34073222.0,-118390965.0,266.0,6.037701e+07,10389.0,3101.0,96086.0,0.0,256538.0,2016.0,9407.01,6.037701e+13,1734
4,11705026,1834372,1.0,2.0,1421.0,6037.0,1.0,33999877.0,-118291863.0,261.0,6.037232e+07,12447.0,3101.0,96018.0,0.0,11982.0,2016.0,543.69,6.037232e+13,1772


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38582 entries, 0 to 38581
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   parcelid                    38582 non-null  int64  
 1   id                          38582 non-null  int64  
 2   bathrooms                   38582 non-null  float64
 3   bedrooms                    38582 non-null  float64
 4   square_feet                 38474 non-null  float64
 5   fips                        38582 non-null  float64
 6   full_baths                  38438 non-null  float64
 7   latitude                    38582 non-null  float64
 8   longitude                   38582 non-null  float64
 9   propertylandusetypeid       38582 non-null  float64
 10  rawcensustractandblock      38582 non-null  float64
 11  city                        37859 non-null  float64
 12  county                      38582 non-null  float64
 13  zip_code                    385

In [11]:
features = df.([])

SyntaxError: invalid syntax (<ipython-input-11-af43867adb35>, line 1)